In [51]:
import numpy as np
import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import glob
import pickle

def camera_calibration():

    nx=9 #chessboard corners in x direction
    ny=6 #chessboard corners in y direction
    #initialize image points and object points
    image_points=[]  #holds image points in 2D plane
    object_points = [] #holds image points in 3D real world space

    objp = np.zeros((ny*nx,3),np.float32)  #initialize the 3D points to all zeros
    objp[:,:2]=np.mgrid[0:nx,0:ny].T.reshape(-1,2)  #set x and y points in real space

    cal_images = glob.glob('./calibration*.jpg')  #fetch all image files for calibration

    for index,fname in enumerate(cal_images):
        cal_image=mpimg.imread(fname)   #read each image file
        
        #convert image to gray
        gray=cv2.cvtColor(cal_image,cv2.COLOR_RGB2GRAY)
        
        #find chessboard corners
        ret,corners = cv2.findChessboardCorners(gray,(nx,ny),None)
        #if corners are found, add object points and image points
        if ret==True:
            object_points.append(objp)
            image_points.append(corners)
            
            #draw and display the corners
            cv2.drawChessboardCorners(cal_image,(nx,ny),corners,ret)
            
            write_name = './/output_images/corners_found' + str(index) + '.jpg'
            #save the drawn chessboard corner image
            cv2.imwrite(write_name,cal_image)
            
    #calibrate camera using image points and object points
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(object_points,image_points,gray.shape[::-1],None,None)

    #save the retrieved camera matrix and distortion coefficients for later reference
    dist_pickle = {}
    dist_pickle["mtx"] = mtx
    dist_pickle["dist"] = dist
    pickle.dump(dist_pickle, open("./calibration_pickle.p", "wb"))
    
    
camera_calibration()   